In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime



inspeccionem que hi ha dins del csv de les dades meteorologiques

In [ ]:
data_allmeteo=pd.read_csv('Dades_meteorol_giques_de_la_XEMA.csv')
data_allmeteo.info()
data_allmeteo.head()


In [ ]:
type(data_allmeteo['DATA_LECTURA'][0])

veiem que la columna data_lectura es un string, ho convertirem a datetime per poder treballar més fàcil

In [ ]:
data_allmeteo['DATA_LECTURA'] = pd.to_datetime(data_allmeteo['DATA_LECTURA'])

In [ ]:
type(data_allmeteo['DATA_LECTURA'][0])

now of all the data i want to keep just the data from 2021 and 2022

In [ ]:
data_allmeteo = data_allmeteo[data_allmeteo['DATA_LECTURA'].dt.year.isin([2021, 2022])] # Filter to only keep data from 2021 and 2022

In [ ]:
data_allmeteo.info()
data_allmeteo.head()

Now we are going to organize the data into variables

In [ ]:
pivot_df = data_allmeteo.pivot(index='DATA_LECTURA', columns='CODI_VARIABLE', values='VALOR_LECTURA')
pivot_df = pivot_df.reset_index()


In [ ]:
# create a dictionary to map the old column names to new column names
column_map = {32: '03.TM',
              40: '04.TX',
              42: '05.TN',
              33: '06.HRM',
              35: '07.PPT24H',
              34: '08.HPA',
              36: '09.RS24H',
              30: '10.VVM10',
              31: '11.DVM10'}

# rename the columns using the dictionary
pivot_df= pivot_df.rename(columns=column_map)
pivot_df=pivot_df[['DATA_LECTURA','03.TM','04.TX','05.TN','06.HRM','07.PPT24H','08.HPA', '09.RS24H','10.VVM10','11.DVM10']]

separete data into 2021 and 2022

In [ ]:
data_meteo_2021_2022=pivot_df.copy()

In [ ]:
data_meteo_2021 = pivot_df[pivot_df['DATA_LECTURA'].dt.year.isin([2021])] # Filter to only keep data from 2021 

In [ ]:
data_meteo_2022 = pivot_df[pivot_df['DATA_LECTURA'].dt.year.isin([2022])] # Filter to only keep data from 2021 

In [ ]:
x=data_allmeteo['DATA_LECTURA'].unique()
print(len(x))
print(len(data_meteo_2021))
print(len(data_meteo_2022))
print(len(data_meteo_2021)+len(data_meteo_2022))

In [ ]:
data_meteo_2021_2022.to_csv('data_meteo_2021_2022.csv', index=False)
data_meteo_2021.to_csv('data_meteo_2021.csv', index=False)
data_meteo_2022.to_csv('data_meteo_2022.csv', index=False)

 aquest pas es un cop fet l'experiment de correlacions entre variables on obtinc el csv 
 amb les variables que m'interesen
 omplo els valors nan de la columna vm10 amb la mitjana

In [92]:
data_meteo_final=pd.read_csv('data_meteo_2021_2022.csv')
data_meteo_final= data_meteo_final[['DATA_LECTURA','03.TM','06.HRM','07.PPT24H','08.HPA', '09.RS24H','10.VVM10']]
data_meteo_final['10.VVM10'].fillna((data_meteo_final['10.VVM10'].mean()), inplace=True)
data_meteo_final['10.VVM10']=data_meteo_final['10.VVM10'].round(1)
data_meteo_final.info()
data_meteo_final.to_csv('data_meteo_estudifinal_2021_2022.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35040 entries, 0 to 35039
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   DATA_LECTURA  35040 non-null  object 
 1   03.TM         35040 non-null  float64
 2   06.HRM        35040 non-null  float64
 3   07.PPT24H     35040 non-null  float64
 4   08.HPA        35040 non-null  float64
 5   09.RS24H      35040 non-null  float64
 6   10.VVM10      35040 non-null  float64
dtypes: float64(6), object(1)
memory usage: 1.9+ MB


In [ ]:
data_meteo_2021_2022.info()


In [ ]:
data_meteo_2021.info()

In [ ]:
data_meteo_2022.info()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

data_meteo_2021_2022 = data_meteo_2021_2022.drop('DATA_LECTURA',1)

plt.figure(figsize=(16,8))
# define the mask to set the values in the upper triangle to True
mask = np.triu(np.ones_like(data_meteo_2021_2022.corr(method ='pearson'), dtype=np.bool))
heatmapP = sns.heatmap(data_meteo_2021_2022.corr(method ='pearson'), mask=mask, vmin=-1, vmax=1, annot=True, cmap='BrBG')
heatmapP.set_title('Taula de correlació entre variables. Pearson', fontdict={'fontsize':15}, pad=16);

plt.savefig('Pearson.png', dpi=300, bbox_inches='tight')

In [ ]:
plt.figure(figsize=(16,8))
# define the mask to set the values in the upper triangle to True
mask = np.triu(np.ones_like(data_meteo_2021_2022.corr(method ='spearman'), dtype=np.bool))
heatmapS = sns.heatmap(data_meteo_2021_2022.corr(method ='spearman'), mask=mask, vmin=-1, vmax=1, annot=True, cmap='BrBG')
heatmapS.set_title('Taula de correlació entre variables. Spearman', fontdict={'fontsize':15}, pad=16);

plt.savefig('Spearman.png', dpi=300, bbox_inches='tight')

In [ ]:
#Calcul dels P-values Spearman de les dades 
from scipy.stats import spearmanr
import pandas as pd

def calculate_pvaluesS(data_meteo_2021_2022):
    data_meteo_2021_2022 = data_meteo_2021_2022.dropna()._get_numeric_data()
    dfcols = pd.DataFrame(columns=data_meteo_2021_2022.columns)
    pvalues = dfcols.transpose().join(dfcols, how='outer')
    for r in data_meteo_2021_2022.columns:
        for c in data_meteo_2021_2022.columns:
            pvalues[r][c] = round(spearmanr(data_meteo_2021_2022[r], data_meteo_2021_2022[c])[1], 4)
    return pvalues


In [ ]:
#Calcul dels P-values Pearson de les dades 
from scipy.stats import pearsonr
import pandas as pd

def calculate_pvaluesP(data_meteo_2021_2022):
    data_meteo_2021_2022 = data_meteo_2021_2022.dropna()._get_numeric_data()
    dfcols = pd.DataFrame(columns=data_meteo_2021_2022.columns)
    pvalues = dfcols.transpose().join(dfcols, how='outer')
    for r in data_meteo_2021_2022.columns:
        for c in data_meteo_2021_2022.columns:
            pvalues[r][c] = round(pearsonr(data_meteo_2021_2022[r], data_meteo_2021_2022[c])[1], 4)
    return pvalues

In [ ]:
#Calcul dels P value de Spearman i Pearson
PvalS=[]
PvalP=[]
PvalS=calculate_pvaluesS(data_meteo_2021_2022).values
PvalP=calculate_pvaluesP(data_meteo_2021_2022).values

#PvalC = PvalC.astype(int)
PvalS=PvalS.astype(float)
PvalS=PvalS.round(5)

PvalP=PvalP.astype(float)
PvalP=PvalP.round(5)

#representarció
plt.figure(figsize=(16,8))
# define the mask to set the values in the upper triangle to True
mask = np.triu(np.ones_like(PvalS, dtype=np.bool))
heatmap = sns.heatmap(PvalS, mask=mask, vmin=-1, vmax=1, annot=True, cmap='BrBG')
heatmap.set_title('PValues Pearson', fontdict={'fontsize':15}, pad=16);
plt.savefig('PvalSpearman.png', dpi=300, bbox_inches='tight')

#representarció
plt.figure(figsize=(16,8))
# define the mask to set the values in the upper triangle to True
mask = np.triu(np.ones_like(PvalP, dtype=np.bool))
heatmap = sns.heatmap(PvalP, mask=mask, vmin=-1, vmax=1, annot=True, cmap='BrBG')
heatmap.set_title('PValues Spearman', fontdict={'fontsize':15}, pad=16);
plt.savefig('PvalPearson.png', dpi=300, bbox_inches='tight')

In [ ]:
data_meteo_2021_2022=data_meteo_2021_2022.dropna()
corr_matrix = data_meteo_2021_2022.corr(method = 'pearson')
from scipy.stats import pearsonr
pvalues = []
for i in range(len(data_meteo_2021_2022.columns)):
    for j in range(i+1, len(data_meteo_2021_2022.columns)):
        corr, p_value = pearsonr(data_meteo_2021_2022.iloc[:,i], data_meteo_2021_2022.iloc[:,j])
        pvalues.append(p_value)
        
significant_corr = []
sig_threshold = 0.05

for index, p_value in enumerate(pvalues):
    if p_value < sig_threshold:
        significant_corr.append((data_meteo_2021_2022.columns[index//len(data_meteo_2021_2022.columns)], data_meteo_2021_2022.columns[index%len(data_meteo_2021_2022.columns)]))

significant_df = pd.DataFrame(significant_corr, columns=['Variable 1', 'Variable 2'])

